In [1]:
import sys
sys.path.append('..')
from src.utilities import *

from src.features.sales_feature_extraction import extract_sales_features

## 2. Master creation

### 2.1 Define cross join of month - region - brand

In [2]:
sales_train = pd.read_csv(os.path.join(raw_path, 'sales_train.csv'))

months = pd.DataFrame({'month': sales_train.month.unique()})
regions = pd.DataFrame({'region': sales_train.region.unique()})
brands = pd.DataFrame({'brand': ['brand_1', 'brand_2']})

months['dummy_col'] = 0
regions['dummy_col'] = 0
brands['dummy_col'] = 0

master = months.merge(regions, how = 'outer', on = 'dummy_col')
master = master.merge(brands, how = 'outer', on = 'dummy_col')
master.drop(columns = 'dummy_col', inplace = True)
master = master.merge(sales_train, how = 'left', on = ['month', 'region', 'brand'])
master.head()

,month,region,brand,sales
0,2020-01,region_0,brand_1,0.0
1,2020-01,region_0,brand_2,0.0
2,2020-01,region_1,brand_1,0.0
3,2020-01,region_1,brand_2,0.0
4,2020-01,region_2,brand_1,0.0


### 2.2 Append features

In [3]:
sales_features = extract_sales_features(sales_train)

master = master.merge(sales_features, how = 'left', on = ['month', 'region', 'brand'])
master.head()

,month,region,brand,sales,sales_univ_b1_1mo,sales_univ_b12_market_1mo,sales_univ_b2_1mo,sales_univ_b3_1mo,sales_univ_b3_market_1mo,sales_univ_1mo,sales_univ_market_1mo,sales_univ_brand_1mo,sales_region_b3_1mo,sales_region_b3market_1mo,sales_region_b12market_1mo,sales_univ_b1_2mo,sales_univ_b1_3mo,sales_univ_b1_4mo,sales_univ_b1_trend_2mo,sales_univ_b1_trend_3mo,sales_univ_b1_trend_4mo,sales_univ_b12_market_2mo,sales_univ_b12_market_3mo,sales_univ_b12_market_4mo,sales_univ_b12_market_trend_2mo,sales_univ_b12_market_trend_3mo,sales_univ_b12_market_trend_4mo,sales_univ_b2_2mo,sales_univ_b2_3mo,sales_univ_b2_4mo,sales_univ_b2_trend_2mo,sales_univ_b2_trend_3mo,sales_univ_b2_trend_4mo,sales_univ_b3_2mo,sales_univ_b3_3mo,sales_univ_b3_4mo,sales_univ_b3_trend_2mo,sales_univ_b3_trend_3mo,sales_univ_b3_trend_4mo,sales_univ_b3_market_2mo,sales_univ_b3_market_3mo,sales_univ_b3_market_4mo,sales_univ_b3_market_trend_2mo,sales_univ_b3_market_trend_3mo,sales_univ_b3_market_trend_4mo,sales_univ_2mo,sales_univ_3mo,sales_univ_4mo,sales_univ_trend_2mo,sales_univ_trend_3mo,sales_univ_trend_4mo,sales_univ_market_2mo,sales_univ_market_3mo,sales_univ_market_4mo,sales_univ_market_trend_2mo,sales_univ_market_trend_3mo,sales_univ_market_trend_4mo,sales_univ_brand_2mo,sales_univ_brand_3mo,sales_univ_brand_4mo,sales_univ_brand_trend_2mo,sales_univ_brand_trend_3mo,sales_univ_brand_trend_4mo,sales_region_b3_2mo,sales_region_b3_3mo,sales_region_b3_4mo,sales_region_b3_trend_2mo,sales_region_b3_trend_3mo,sales_region_b3_trend_4mo,sales_region_b3market_2mo,sales_region_b3market_3mo,sales_region_b3market_4mo,sales_region_b3market_trend_2mo,sales_region_b3market_trend_3mo,sales_region_b3market_trend_4mo,sales_region_b12market_2mo,sales_region_b12market_3mo,sales_region_b12market_4mo,sales_region_b12market_trend_2mo,sales_region_b12market_trend_3mo,sales_region_b12market_trend_4mo
0,2020-01,region_0,brand_1,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-01,region_0,brand_2,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-01,region_1,brand_1,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-01,region_1,brand_2,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-01,region_2,brand_1,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### 2.3 Save master table

In [4]:
master.to_csv(os.path.join(processed_path, 'master.csv'), index = False)